<a href="https://colab.research.google.com/github/rileighdethy/mgmt467-analytics-portfolio/blob/main/Lab_VertexAI_BigQuery_PromptsOnly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab: Vertex AI–Assisted BigQuery Analytics — Example Prompts
**Goal:** Practice moving from simple SQL to complex analytics in BigQuery using *only* carefully engineered prompts with Vertex AI (Gemini).  
**Important:** This notebook contains **prompts only** (no starter code). Paste the prompts into **Vertex AI Studio**, **Vertex AI in Colab Enterprise**, or your chosen chat interface, and then run the generated SQL directly in **BigQuery**. If you decide to automate later, you can ask Vertex AI to convert the winning SQL into a Colab pipeline.

## How to use this prompts-only notebook
1. Open **Vertex AI Studio** (or Gemini in Colab Enterprise chat panel).  
2. Copy a prompt from this notebook and paste it into the model. Do **not** paste any code from here; let the model generate it.  
3. Run the generated SQL in **BigQuery** (Console → BigQuery Studio).  
4. Iterate: refine the prompt when results aren’t what you expect.  
5. Document: capture your final SQL, plus a one-sentence takeaway, in your notes/README.

## Dataset assumptions
Use one of these sources (adjust table paths accordingly):
- **Global Superstore (Kaggle)** loaded into BigQuery (e.g., `[YOUR_PROJECT].superstore_data.sales`)  
- **TheLook eCommerce** public dataset: `bigquery-public-data.thelook_ecommerce`  
If you are using *Global Superstore*, make sure column names match your schema (e.g., `Order_Date`, `Region`, `Category`, `Sub_Category`, `Sales`, `Profit`, `Discount`, `State`, `Customer_ID`, `Ship_Mode`).

---
## Prompting guardrails (quick checklist)
- **Be explicit**: table path, column names, filters, output columns, sort order, and limits.  
- **Ask for runnable SQL**: “Return a BigQuery SQL block only.”  
- **Control cost**: ask for `LIMIT` during exploration and remove it for the final run.  
- **Validate**: request a brief explanation of why each clause is present and how you can sanity-check results.
---

## Install Dependencies

In [ ]:
# Install the Google Cloud BigQuery client library
!pip install google-cloud-bigquery==3.17.0 pandas==2.1.4

In [ ]:
# Authenticate your Colab environment
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


## Copy Schema to a dataframe

In [ ]:
from google.cloud import bigquery
import pandas as pd

# Replace with your Google Cloud Project ID
project_id = 'mgmt-467-471613' # This is derived from your provided table name
dataset_id = 'superstore_data'
table_id = 'sales_loaded'

# Construct a BigQuery client object.
client = bigquery.Client(project=project_id)

# Get the table object
table_ref = client.dataset(dataset_id).table(table_id)
table = client.get_table(table_ref)

# Extract schema information
schema_list = []
for field in table.schema:
    schema_list.append({
        'name': field.name,
        'field_type': field.field_type,
        'mode': field.mode,
        'description': field.description
    })

# Convert to Pandas DataFrame
schema_df = pd.DataFrame(schema_list)

# Display the schema DataFrame (optional, for verification)
print("Schema DataFrame created:")
schema_df
# To see the output, run the code.


Schema DataFrame created:


,name,field_type,mode,description
0,Category,STRING,NULLABLE,None
1,City,STRING,NULLABLE,None
2,Country,STRING,NULLABLE,None
3,Customer ID,STRING,NULLABLE,None
4,Customer Name,STRING,NULLABLE,None
5,Discount,FLOAT,NULLABLE,None
6,Market,STRING,NULLABLE,None
7,Record Count,INTEGER,NULLABLE,None
8,Order Date,DATETIME,NULLABLE,None
9,Order ID,STRING,NULLABLE,None


## CLean Column Names

In [ ]:
# --- 1. Clean the Column Names ---
# Create a 'clean_name' column with standard naming conventions:
# lowercase, with spaces and hyphens replaced by underscores.
schema_df['clean_name'] = schema_df['name'].str.lower().str.replace(' ', '_').str.replace('-', '_')


# --- 2. Generate the Aliases for the SELECT Clause ---
column_expressions = []
for index, row in schema_df.iterrows():
    original_name = row['name']
    clean_name = row['clean_name']

    # If the original name contains a space or special character, it needs to be
    # enclosed in backticks (`) in the SQL statement.
    if ' ' in original_name or '-' in original_name:
        expression = f'`{original_name}` AS {clean_name}'
    else:
        # If the name is already clean, we still alias it for consistency.
        expression = f'{original_name} AS {clean_name}'
    column_expressions.append(expression)

# Join all the individual column expressions into a single, formatted string.
select_clause = ",\n  ".join(column_expressions)


# --- 3. Construct the Final CREATE VIEW Statement ---
new_view_id = 'superstore_clean' # You can change this if you like

create_view_sql = f"""
CREATE OR REPLACE VIEW `{project_id}.{dataset_id}.{new_view_id}` AS
SELECT
  {select_clause}
FROM
  `{project_id}.{dataset_id}.{table_id}`;
"""

# --- 4. Print the Final SQL ---
print("--- Copy the SQL below and run it in your BigQuery Console ---")
print(create_view_sql)

--- Copy the SQL below and run it in your BigQuery Console ---

CREATE OR REPLACE VIEW `mgmt-467-471613.superstore_data.superstore_clean` AS
SELECT
  Category AS category,
  City AS city,
  Country AS country,
  `Customer ID` AS customer_id,
  `Customer Name` AS customer_name,
  Discount AS discount,
  Market AS market,
  `Record Count` AS record_count,
  `Order Date` AS order_date,
  `Order ID` AS order_id,
  `Order Priority` AS order_priority,
  `Product ID` AS product_id,
  `Product Name` AS product_name,
  Profit AS profit,
  Quantity AS quantity,
  Region AS region,
  `Row ID` AS row_id,
  Sales AS sales,
  Segment AS segment,
  `Ship Date` AS ship_date,
  `Ship Mode` AS ship_mode,
  `Shipping Cost` AS shipping_cost,
  State AS state,
  `Sub-Category` AS sub_category,
  Year AS year,
  Market2 AS market2,
  weeknum AS weeknum
FROM
  `mgmt-467-471613.superstore_data.sales_loaded`;



## Generate View with standard column naming convention

In [ ]:
# Execute the CREATE VIEW SQL query
try:
    query_job = client.query(create_view_sql)  # API request
    query_job.result()  # Waits for the query to finish
    print(f"View '{new_view_id}' created/replaced successfully in dataset '{dataset_id}'.")
except Exception as e:
    print(f"An error occurred while creating the view: {e}")

# Now, let's print 10 rows from the newly created view to verify
print(f"\n--- First 10 rows from the new view '{new_view_id}' ---")
try:
    # Construct a reference to the new view
    view_table_ref = client.dataset(dataset_id).table(new_view_id)

    # Fetch the first 10 rows
    rows = client.list_rows(view_table_ref, max_results=10)

    # Print header
    print(" | ".join([field.name for field in rows.schema]))
    print("-" * 80) # Separator

    # Print rows
    for row in rows:
        print(" | ".join([str(item) for item in row.values()]))

except Exception as e:
    print(f"An error occurred while fetching rows from the view: {e}")



View 'superstore_clean' created/replaced successfully in dataset 'superstore_data'.

--- First 10 rows from the new view 'superstore_clean' ---
category | city | country | customer_id | customer_name | discount | market | record_count | order_date | order_id | order_priority | product_id | product_name | profit | quantity | region | row_id | sales | segment | ship_date | ship_mode | shipping_cost | state | sub_category | year | market2 | weeknum
--------------------------------------------------------------------------------
An error occurred while fetching rows from the view: 400 GET https://bigquery.googleapis.com/bigquery/v2/projects/mgmt-467-471613/datasets/superstore_data/tables/superstore_clean/data?maxResults=10&formatOptions.useInt64Timestamp=True&prettyPrint=false: Cannot list a table of type VIEW.


In [ ]:
# This assumes your 'client' object from the previous cell is still active
# and correctly authenticated.

print("✅ Step 1: Defining the query string...")

query_string = """
SELECT
  order_id,
  customer_name,
  product_name,
  sales,
  profit
FROM
  `mgmt-467-471613.superstore_data.superstore_clean`
LIMIT 10;
"""

print("✅ Step 2: Sending the query to BigQuery. This may take a moment...")

# Use a try-except block to catch potential errors
try:
    query_job = client.query(query_string)

    print("✅ Step 3: Waiting for query to complete and fetching results...")
    results_df = query_job.to_dataframe()

    print(f"✅ Step 4: Query finished. Found {len(results_df)} rows.")

    if results_df.empty:
        print("\n⚠️ The query ran successfully but returned an empty result. Please double-check that your 'superstore_clean' view exists and the original table has data.")
    else:
        print("\n--- Displaying Results ---")
        display(results_df)

except Exception as e:
    print(f"\n❌ An error occurred: {e}")

✅ Step 1: Defining the query string...
✅ Step 2: Sending the query to BigQuery. This may take a moment...
✅ Step 3: Waiting for query to complete and fetching results...
✅ Step 4: Query finished. Found 10 rows.

--- Displaying Results ---


,order_id,customer_name,product_name,sales,profit
0,ID-2012-35556,Brian Dahlen,"Acme Scissors, Easy Grip",244,104.49
1,ID-2012-35556,Brian Dahlen,"Cameo Interoffice Envelope, with clear poly wi...",204,24.36
2,IN-2014-29767,Cathy Armstrong,"Epson Receipt Printer, White",346,13.77
3,IN-2014-29767,Cathy Armstrong,"Ikea Library with Doors, Mobile",732,102.42
4,IN-2014-29767,Cathy Armstrong,"Rubbermaid Door Stop, Erganomic",170,79.68
5,IN-2013-43151,Stephanie Ulpright,"OIC Paper Clips, 12 Pack",14,2.34
6,IN-2011-28920,Ricardo Sperren,"SanDisk Parchment Paper, 8.5 x 11",83,8.28
7,IN-2013-27695,Cari Sayre,"Smead Trays, Wire Frame",333,59.85
8,IN-2014-75456,Allen Armold,"Eaton Memo Slips, Multicolor",15,4.68
9,IN-2014-75456,Allen Armold,"Hoover Toaster, Red",669,280.80


## Part A — SQL Warm‑Up (SELECT, WHERE, ORDER BY, LIMIT, DISTINCT)
**Aim:** Build confidence with precise, unambiguous prompts that yield clean, runnable SQL.

### A1. Unique values (DISTINCT)
**Prompt (paste in Vertex AI):**
```
Act as a senior BigQuery analyst. Produce a **single runnable BigQuery SQL** (no commentary) for:
- Task: List all unique `Sub_Category` values sold in the 'West' region.
- Table: `mgmt-467-47888.lab1_foundation.superstore`
- Filter: `Region = 'West'`
- Output: a single column named `Sub_Category`
- Sort: alphabetically A→Z
- Add: `LIMIT 100` to control cost during exploration.
```
**Reflection:** Did the result match your expectations? If not, what ambiguity in your prompt might have caused the mismatch?

In [ ]:
query_string = """
SELECT
    DISTINCT Sub_Category
FROM
    `mgmt-467-471613.superstore_data.superstore_clean`
WHERE
    Region = 'West'
ORDER BY
    Sub_Category ASC
LIMIT 100;
"""
query_job = client.query(query_string)
results_df = query_job.to_dataframe()
display(results_df)

,Sub_Category
0,Accessories
1,Appliances
2,Art
3,Binders
4,Bookcases
5,Chairs
6,Copiers
7,Envelopes
8,Fasteners
9,Furnishings


### A2. Top‑N by metric (ORDER BY … DESC)
**Prompt:**
```
BigQuery SQL only.
Task: Return the top 10 customers by total profit.
Table: `mgmt-467-47888.lab_foundation.superstore`
Columns used: `Customer_ID`, `Profit`
Output columns: `Customer_ID`, `total_profit`
Logic: SUM Profit per customer, order by `total_profit` DESC
Add `LIMIT 10`.
```
**Tip:** If your schema uses different identifiers (e.g., `Customer Name`), restate column names explicitly.

In [ ]:
query_string_a2 = """
SELECT
    customer_id,
    SUM(profit) AS total_profit
FROM
    `mgmt-467-471613.superstore_data.superstore_clean`
GROUP BY
    customer_id
ORDER BY
    total_profit DESC
LIMIT 10;
"""

query_job_a2 = client.query(query_string_a2)
results_df_a2 = query_job_a2.to_dataframe()
display(results_df_a2)

,customer_id,total_profit
0,TC-209804,8981.3239
1,RB-193604,6976.0959
2,SC-200954,5757.4119
3,HL-150404,5622.4292
4,AB-101054,5444.8055
5,SP-209202,4974.5130
6,TA-213854,4703.7883
7,CA-127751,4045.8780
8,PJ-188352,3986.0040
9,CM-123854,3899.8904


### A3. Basic filtering (WHERE) + sanity checks
**Prompt:**
```
BigQuery SQL only.
Task: Count orders shipped with each `Ship_Mode`, but only for orders in the 'Technology' category.
Table: `[YOUR_PROJECT].superstore_data.sales`
Output: `Ship_Mode`, `order_count`
Logic: COUNT(*) grouped by `Ship_Mode`
Sort by `order_count` DESC
```
**Validation ask:** “Also list two quick sanity checks to verify the numbers.”
```
1. Check the total number of orders where the category equal technology
2. Check a specific ship mode and query it to sum where the category is technology
```

In [ ]:
query_string_a3 = """
SELECT
    ship_mode,
    COUNT(*) AS order_count
FROM
    `mgmt-467-471613.superstore_data.superstore_clean`
WHERE
    category = 'Technology'
GROUP BY
    ship_mode
ORDER BY
    order_count DESC;
"""
query_job_a3 = client.query(query_string_a3)
results_df_a3 = query_job_a3.to_dataframe()
display(results_df_a3)

,ship_mode,order_count
0,Standard Class,6117
1,Second Class,2030
2,First Class,1476
3,Same Day,518


## Part B — Grouped Analytics (GROUP BY, HAVING)
**Aim:** Turn raw facts into grouped metrics and filtered aggregations.

### B1. KPI aggregation with WHERE + GROUP BY
**Prompt:**
```
BigQuery SQL only.
Task: Compute monthly revenue for the last 12 full months.
Table: `[YOUR_PROJECT].superstore_data.sales`
Assume: `Order_Date` is a DATE or TIMESTAMP column named exactly `Order_Date`.
Output: `year_month` (YYYY-MM format), `monthly_revenue`
Logic: Truncate date to month, SUM `Sales`, filter to last 12 full months.
Sort by `year_month` ascending.
Include a `LIMIT` safeguard for exploration.
```

In [ ]:
query_string_b1 = """
WITH monthly_sales AS (
    SELECT
        DATE_TRUNC(DATE(order_date), MONTH) AS order_month,
        SUM(Sales) AS monthly_revenue
    FROM
        `mgmt-467-471613.superstore_data.superstore_clean`
    WHERE
        order_date IS NOT NULL
    GROUP BY
        order_month
),
ranked_sales AS (
    SELECT
        FORMAT_DATE('%Y-%m', order_month) AS year_month,
        monthly_revenue,
        RANK() OVER (ORDER BY order_month DESC) as month_rank
    FROM
        monthly_sales
)
SELECT
    year_month,
    monthly_revenue
FROM
    ranked_sales
WHERE
    month_rank <= 12
ORDER BY
    year_month ASC;
"""

query_job_b1 = client.query(query_string_b1)
results_df_b1 = query_job_b1.to_dataframe()
display(results_df_b1)

,year_month,monthly_revenue
0,2024-10,67569
1,2024-11,60818
2,2024-12,75860
3,2025-01,70892
4,2025-02,56394
5,2025-03,63714
6,2025-04,85495
7,2025-05,55720
8,2025-06,63666
9,2025-07,64118


### B2. Post‑aggregation filter (HAVING)
**Prompt:**
```
BigQuery SQL only.
Task: Find sub-categories whose total profit over the entire dataset is negative.
Table: `[YOUR_PROJECT].superstore_data.sales`
Output: `Sub_Category`, `total_profit`
Logic: SUM `Profit` GROUP BY `Sub_Category`, HAVING SUM(Profit) < 0
Sort by `total_profit` ASC (most negative first).
```
**Why HAVING?** Ask the model to include a 1-sentence explanation of why HAVING is used instead of WHERE here.

HAVING is used because you are filtering based on an aggregated vale (SUM(profit)), which is calculated after the data has been grouped, whereas WHERE is used to filter individual rows before any grouping occurs


In [ ]:
query_string_b2 = """
SELECT
    sub_category,
    SUM(profit) AS total_profit
FROM
    `mgmt-467-471613.superstore_data.superstore_clean`
GROUP BY
    sub_category
HAVING
    SUM(profit) < 0
ORDER BY
    total_profit ASC;
"""

query_job = client.query(query_string_b2)
results_df_b2 = query_job.to_dataframe()
display(results_df_b2)

,sub_category,total_profit
0,Tables,-64083.3887


## Part C — Joins (dimension enrichment)
**Aim:** Use joins to enhance facts with attributes.

### C1. Join facts to a small dimension
*(If you have a customer or product dimension in your schema, use it. Otherwise, request a synthetic example.)*  
**Prompt:**
```
BigQuery SQL only.
Task: Join the sales table to a product dimension to report `Product_ID`, `Product_Name`, and total sales.
Tables: `[YOUR_PROJECT].superstore_data.sales` as s, `[YOUR_PROJECT].superstore_data.products` as p
Join key: `s.Product_ID = p.Product_ID`
Output: `Product_ID`, `Product_Name`, `total_sales`
Sort by `total_sales` DESC
```
**If you lack a dimension table:** Ask the model how to simulate one temporarily via a CTE.

In [ ]:
# This query reports product_id, product_name, and total sales by aggregating from the superstore_clean view.
query_string_c1 = """
SELECT
    product_id,
    product_name,
    SUM(sales) AS total_sales
FROM
    `mgmt-467-471613.superstore_data.superstore_clean`
GROUP BY
    product_id,
    product_name
ORDER BY
    total_sales DESC
LIMIT 100;
"""

query_job_c1 = client.query(query_string_c1)
results_df_c1 = query_job_c1.to_dataframe()
display(results_df_c1)

,product_id,product_name,total_sales
0,TEC-CO-10004722,Canon imageCLASS 2200 Advanced Copier,61600
1,TEC-PH-10004664,"Nokia Smart Phone, with Caller ID",30042
2,OFF-BI-10003527,Fellowes PB500 Electric Punch Plastic Comb Bin...,27454
3,TEC-MA-10002412,Cisco TelePresence System EX90 Videoconferenci...,22638
4,TEC-PH-10004823,"Nokia Smart Phone, Full Size",22261
...,...,...,...
95,FUR-BO-10004999,"Safco Classic Bookcase, Metal",10372
96,TEC-MOT-10003050,"Motorola Smart Phone, Cordless",10350
97,TEC-CO-10002035,"Canon Wireless Fax, High-Speed",10260
98,TEC-AC-10003832,LogitechÂ P710e Mobile Speakerphone,10194


## Part D — Common Table Expressions (CTEs)
**Aim:** Make complex logic readable and testable in steps.

### D1. Multi‑step ranking with CTEs
**Prompt:**
```
BigQuery SQL only.
Goal: Within each `Region`, rank states by total sales and return top 3 per region.
Table: `[YOUR_PROJECT].superstore_data.sales`
CTE 1 (`state_sales`): SUM(Sales) by `Region`, `State`
CTE 2 (`ranked_state_sales`): Add `RANK() OVER (PARTITION BY Region ORDER BY total_sales DESC)` as `sales_rank`
Final SELECT: rows where `sales_rank <= 3`
Output columns: `Region`, `State`, `total_sales`, `sales_rank`
Sort: by `Region`, then `sales_rank`
```
**Ask for**: a one-paragraph explanation of each step, then **provide only the final runnable SQL**.
Here is a breakdown of the SQL query, explaining each step:

CTE 1 (state_sales): This first Common Table Expression calculates the total sales for each combination of region and state. It groups the data from the superstore_clean view by both region and state and then sums the sales for each group, aliasing the result as total_sales. This provides an intermediate result set showing the total sales generated within each state across all regions.

CTE 2 (ranked_state_sales): This second CTE takes the results from the state_sales CTE and adds a ranking based on total sales within each region. It uses the RANK() OVER (PARTITION BY region ORDER BY total_sales DESC) window function. PARTITION BY region divides the data into partitions for each unique region, and ORDER BY total_sales DESC orders the states within each partition by their total sales in descending order. RANK() assigns a rank to each state within its region based on this order, with the highest sales getting rank 1. The result is aliased as sales_rank.

Final SELECT statement: This final part of the query selects the region, state, total_sales, and sales_rank columns from the ranked_state_sales CTE. The WHERE sales_rank <= 3 clause filters these results to keep only the rows where the sales_rank is 3 or less, effectively selecting the top 3 states by total sales within each region. Finally, the results are ordered first by region and then by sales_rank to present the top states for each region in a clear, ranked order.

In [ ]:
# This query ranks states by total sales within each region and returns the top 3
# using Common Table Expressions (CTEs).
query_string_d1 = """
WITH state_sales AS (
    SELECT
        region,
        state,
        SUM(sales) AS total_sales
    FROM
        `mgmt-467-471613.superstore_data.superstore_clean`
    GROUP BY
        region,
        state
),
ranked_state_sales AS (
    SELECT
        region,
        state,
        total_sales,
        RANK() OVER (PARTITION BY region ORDER BY total_sales DESC) AS sales_rank
    FROM
        state_sales
)
SELECT
    region,
    state,
    total_sales,
    sales_rank
FROM
    ranked_state_sales
WHERE
    sales_rank <= 3
ORDER BY
    region,
    sales_rank;
"""
query_job_d1 = client.query(query_string_d1)
results_df_d1 = query_job_d1.to_dataframe()
display(results_df_d1)

,region,state,total_sales,sales_rank
0,Africa,Gauteng,51608,1
1,Africa,Kinshasa,42533,2
2,Africa,Al Qahirah,38436,3
3,Canada,Ontario,35451,1
4,Canada,Quebec,10928,2
5,Canada,British Columbia,9546,3
6,Caribbean,Santo Domingo,78710,1
7,Caribbean,Santiago de Cuba,32456,2
8,Caribbean,Granma,16997,3
9,Central,Ile-de-France,317818,1


### D2. Time‑boxed “most improved” analysis
**Prompt:**
```
BigQuery SQL only.
Goal: Identify the top 5 sub-categories with the largest YoY revenue increase from 2023 to 2024.
Table: `[YOUR_PROJECT].superstore_data.sales`
CTE `yr_sales`: SUM(Sales) by `Sub_Category` and `year` extracted from `Order_Date`
Final: pivot or self-join to compute delta (2024 minus 2023) as `yoy_delta`
Output: `Sub_Category`, `sales_2023`, `sales_2024`, `yoy_delta`
Order by `yoy_delta` DESC
Limit 5
```
**Validation:** Ask the model for two quick failure modes (e.g., missing years) and how to handle them.

In [ ]:
query_string_d2 = """
WITH yr_sales AS (
    SELECT
        sub_category,
        EXTRACT(YEAR FROM order_date) AS sale_year,
        SUM(sales) AS total_sales
    FROM
        `mgmt-467-471613.superstore_data.superstore_clean`
    WHERE
        order_date IS NOT NULL AND
        EXTRACT(YEAR FROM order_date) IN (2023, 2024)
    GROUP BY
        sub_category,
        sale_year
)
SELECT
    ys_2023.sub_category,
    ys_2023.total_sales AS sales_2023,
    ys_2024.total_sales AS sales_2024,
    (ys_2024.total_sales - ys_2023.total_sales) AS yoy_delta
FROM
    yr_sales ys_2023
JOIN
    yr_sales ys_2024
ON
    ys_2023.sub_category = ys_2024.sub_category
WHERE
    ys_2023.sale_year = 2023
    AND ys_2024.sale_year = 2024
ORDER BY
    yoy_delta DESC
LIMIT 5;
"""

query_job_d2 = client.query(query_string_d2)
results_df_d2 = query_job_d2.to_dataframe()
display(results_df_d2)

,sub_category,sales_2023,sales_2024,yoy_delta
0,Copiers,92051,108900,16849
1,Furnishings,20271,30025,9754
2,Appliances,46504,56032,9528
3,Chairs,91081,98646,7565
4,Binders,34193,39118,4925


## Part E — Window Functions (ROW_NUMBER, RANK, DENSE_RANK, LAG/LEAD, moving averages)
**Aim:** Compare rows across partitions and time; compute trends and ranks without collapsing rows.

### E1. Top product per region (ROW_NUMBER)
**Prompt:**
```
BigQuery SQL only.
Task: For each `Region`, return only the single highest-revenue `Sub_Category`.
Table: `[YOUR_PROJECT].superstore_data.sales`
CTE `subcat_sales`: SUM(Sales) by `Region`, `Sub_Category`
Add `ROW_NUMBER() OVER (PARTITION BY Region ORDER BY total_sales DESC)` as rn
Final: filter `rn = 1`
Output: `Region`, `Sub_Category`, `total_sales`
Sort by `Region`
```
**Why `ROW_NUMBER` instead of `RANK`?** Ask the model to add a 2-sentence contrast.

ROW_NUMBER() guarantees a unique rank for each row, ensuring only one top result per region even with ties, whereas RANK() assigns the same rank to tied rows, potentially returning multiple top results. Therefore, ROW_NUMBER() is used here to specifically select the single highest-revenue sub-category per region.

In [ ]:
# This query finds the single highest-revenue sub-category for each region
# using ROW_NUMBER().
query_string_e1 = """
WITH subcat_sales AS (
    SELECT
        region,
        sub_category,
        SUM(sales) AS total_sales
    FROM
        `mgmt-467-471613.superstore_data.superstore_clean`
    GROUP BY
        region,
        sub_category
),
ranked_subcat_sales AS (
    SELECT
        region,
        sub_category,
        total_sales,
        ROW_NUMBER() OVER (PARTITION BY region ORDER BY total_sales DESC) AS rn
    FROM
        subcat_sales
)
SELECT
    region,
    sub_category,
    total_sales
FROM
    ranked_subcat_sales
WHERE
    rn = 1
ORDER BY
    region;
"""
query_job_e1 = client.query(query_string_e1)
results_df_e1 = query_job_e1.to_dataframe()
display(results_df_e1)

,region,sub_category,total_sales
0,Africa,Phones,114830
1,Canada,Storage,10586
2,Caribbean,Copiers,49999
3,Central,Phones,370215
4,Central Asia,Phones,132715
5,EMEA,Phones,114521
6,East,Phones,100628
7,North,Phones,180419
8,North Asia,Bookcases,130070
9,Oceania,Chairs,170286


### E2. YoY growth with LAG
**Prompt:**
```
BigQuery SQL only.
Task: Compute year-over-year revenue growth for 'Phones' sub-category.
Table: `[YOUR_PROJECT].superstore_data.sales`
Steps:
- Filter to `Sub_Category = 'Phones'`
- Aggregate yearly revenue using EXTRACT(YEAR FROM Order_Date)
- Add `LAG(yearly_revenue) OVER (ORDER BY year)` as `prev_revenue`
- Compute `yoy_pct = 100.0 * (yearly_revenue - prev_revenue) / prev_revenue`
Output: `year`, `yearly_revenue`, `prev_revenue`, `yoy_pct`
Sort by `year` ASC
```
**Ask for**: a guard against divide-by-zero or NULL previous year.

In [ ]:
# This query computes year-over-year revenue growth for the 'Phones' sub-category
# using the LAG window function and handling potential NULL values or divide-by-zero.
query_string_e2 = """
WITH yearly_sales AS (
    SELECT
        sub_category,
        EXTRACT(YEAR FROM order_date) AS sale_year,
        SUM(sales) AS total_sales
    FROM
        `mgmt-467-471613.superstore_data.superstore_clean`
    WHERE
        sub_category = 'Phones' AND order_date IS NOT NULL
    GROUP BY
        sub_category,
        sale_year
),
lagged_sales AS (
    SELECT
        sale_year,
        total_sales AS yearly_revenue,
        LAG(total_sales) OVER (ORDER BY sale_year) AS prev_revenue
    FROM
        yearly_sales
)
SELECT
    sale_year,
    yearly_revenue,
    prev_revenue,
    SAFE_DIVIDE(100.0 * (yearly_revenue - prev_revenue), prev_revenue) AS yoy_pct
FROM
    lagged_sales
ORDER BY
    sale_year ASC;
"""

query_job_e2 = client.query(query_string_e2)
results_df_e2 = query_job_e2.to_dataframe()
display(results_df_e2)

,sale_year,yearly_revenue,prev_revenue,yoy_pct
0,2010,106826,<NA>,NaN
1,2011,100625,106826,-5.804767
2,2012,112756,100625,12.055652
3,2013,91679,112756,-18.692575
4,2014,106234,91679,15.876046
5,2015,100764,106234,-5.149011
6,2016,100690,100764,-0.073439
7,2017,112464,100690,11.693316
8,2018,99168,112464,-11.822450
9,2019,111414,99168,12.348742


### E3. 3‑month moving average (MA)
**Prompt:**
```
BigQuery SQL only.
Task: For the 'Corporate' segment, compute a 3-month moving average of monthly revenue.
Table: `[YOUR_PROJECT].superstore_data.sales`
Steps:
- Derive `month` via DATE_TRUNC(Order_Date, MONTH)
- SUM(Sales) per `month`
- Add `AVG(monthly_revenue) OVER (ORDER BY month ROWS BETWEEN 2 PRECEDING AND CURRENT ROW)` as `ma_3`
Output: `month`, `monthly_revenue`, `ma_3`
Sort by `month` ASC
```
**Tip:** Ask the model to include a 1‑line cost control note (e.g., restrict date range while iterating).

In [ ]:
# This query computes a 3-month moving average of monthly revenue for the 'Corporate' segment.
# Cost Control Tip: Restrict the date range in the WHERE clause during exploration to reduce scanned bytes.
query_string_e3 = """
WITH monthly_sales_corporate AS (
    SELECT
        DATE_TRUNC(order_date, MONTH) AS month,
        SUM(sales) AS monthly_revenue
    FROM
        `mgmt-467-471613.superstore_data.superstore_clean`
    WHERE
        segment = 'Corporate' AND order_date IS NOT NULL
        -- AND order_date BETWEEN '2024-01-01' AND '2024-12-31' -- Example: uncomment and adjust dates for testing
    GROUP BY
        month
),
moving_average AS (
    SELECT
        month,
        monthly_revenue,
        AVG(monthly_revenue) OVER (ORDER BY month ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) AS ma_3
    FROM
        monthly_sales_corporate
)
SELECT
    FORMAT_DATE('%Y-%m', month) AS month,
    monthly_revenue,
    ma_3
FROM
    moving_average
ORDER BY
    month ASC;
"""

query_job_e3 = client.query(query_string_e3)
results_df_e3 = query_job_e3.to_dataframe()
display(results_df_e3)

,month,monthly_revenue,ma_3
0,2010-01,22244,22244.000000
1,2010-02,15790,19017.000000
2,2010-03,20530,19521.333333
3,2010-04,17076,17798.666667
4,2010-05,22957,20187.666667
...,...,...,...
184,2025-05,16838,20410.333333
185,2025-06,16151,18374.000000
186,2025-07,23147,18712.000000
187,2025-08,24063,21120.333333


## Part F — Debugging & Optimization Prompts
**Aim:** Use the model as a rubber duck for error handling and performance.

### F1. Explain the error, propose a fix
**Prompt:**
```
Error Message:
BadRequest: 400 No matching signature for function PARSE_DATE Argument types: STRING, DATETIME Signature: PARSE_DATE(STRING, STRING) Argument 2: Unable to coerce type DATETIME to expected type STRING at [9:9]; reason: invalidQuery, location: query, message: No matching signature for function PARSE_DATE Argument types: STRING, DATETIME Signature: PARSE_DATE(STRING, STRING) Argument 2: Unable to coerce type DATETIME to expected type STRING at [9:9] Location: US Job ID: 65b9b91a-3a9e-4ac6-bb6b-fa36a7cae1e9
```
Act as a BigQuery trouble‑shooter.
1) Identify the root cause.
The issue was caused by thr PARSE_DATE being used incorrectly from when my order_date and ship_date was incorrectly uploaded to my big query
2) Propose the smallest possible fix.
I can just use EXTRACT(YEAR FROM order_date) since order date is already in a date/datetime format that BigQuery can extract the year from directly
3) Suggest a quick sanity check query to verify the fix.
Make sure that the dates are actually pulling

Return only the corrected SQL and a 2‑sentence rationale.

The code is corrected in cell be2c98f6. I know that the code was corrected because it had an output, compared to the code with an error which did not have an output.


### F2. Reduce cost / improve speed
**Prompt:**
```
Act as a BigQuery cost optimizer.
Given this query (below), list 3 ways to reduce scanned bytes and improve performance without changing the business logic.
WITH state_sales AS (
    SELECT
        region,
        state,
        SUM(sales) AS total_sales
    FROM
        `mgmt-467-471613.superstore_data.superstore_clean`
    GROUP BY
        region,
        state
),
ranked_state_sales AS (
    SELECT
        region,
        state,
        total_sales,
        RANK() OVER (PARTITION BY region ORDER BY total_sales DESC) AS sales_rank
    FROM
        state_sales
)
SELECT
    region,
    state,
    total_sales,
    sales_rank
FROM
    ranked_state_sales
WHERE
    sales_rank <= 3
ORDER BY
    region,
    sales_rank;
Prioritize: partition filters, column pruning, pre-aggregations, and temporary results via CTEs.
```

Gemini Answer:
1. **Partition Filtering**: If the superstore_clean table is partitioned by region, you can add a WHERE region IN (...) clause in the initial state_sales CTE to only scan data for the regions you are interested in, significantly reducing scanned bytes if you only need a subset of regions.
2. **Column Pruning**: Ensure that only the necessary columns (region, state, and sales) are selected in the initial state_sales CTE. The current query already does a good job of this by only selecting these columns.
3. **Pre-aggregations**: (if applicable): If this query is run frequently and the data doesn't change often, consider creating a materialized view that pre-calculates the state_sales CTE. This would mean subsequent queries on the materialized view would read less data than scanning the base table every time. However, this adds complexity for managing the materialized view.

## Part G — Validation & Counter‑examples (DIVE: Validate)
**Aim:** Avoid “first‑answer fallacy” by testing alternatives.

### G1. Ask for counter‑queries
**Prompt:**
```
I concluded that 'Tables' is a high‑sales but negative‑profit sub-category due to high discounts.
Create two alternative BigQuery SQL queries that could falsify or nuance this finding:
- One that slices by region and time
- One that controls for order priority or ship mode
Return BigQuery SQL only, then a one-paragraph note on how to compare outcomes.
```

# Query 1: Slices by region and year
```
query_string_g1_region_time = """
SELECT
    region,
    EXTRACT(YEAR FROM order_date) AS sale_year,
    SUM(sales) AS total_sales,
    SUM(profit) AS total_profit,
    SUM(discount) AS total_discount
FROM
    `mgmt-467-471613.superstore_data.superstore_clean`
WHERE
    sub_category = 'Tables'
    AND order_date IS NOT NULL
GROUP BY
    region,
    sale_year
ORDER BY
    region,
    sale_year;
"""
```
# Query 2: Controls for order priority or ship mode
```
query_string_g1_order_ship = """
SELECT
    order_priority,
    ship_mode,
    SUM(sales) AS total_sales,
    SUM(profit) AS total_profit,
    SUM(discount) AS total_discount
FROM
    `mgmt-467-471613.superstore_data.superstore_clean`
WHERE
    sub_category = 'Tables'
GROUP BY
    order_priority,
    ship_mode
ORDER BY
    order_priority,
    ship_mode;
"""

print("--- Query 1: Slices by region and year ---")
print(query_string_g1_region_time)

print("\n--- Query 2: Controls for order priority or ship mode ---")
print(query_string_g1_order_ship)
```

To compare outcomes: Run each query in BigQuery and examine the results. The first query will show if the negative profitability of 'Tables' is consistent across different regions and over time. If some regions or years show positive profit, it nuances the overall finding. The second query will reveal if specific order priorities or ship modes are disproportionately contributing to the negative profit, potentially highlighting operational or pricing issues tied to how 'Tables' are shipped or ordered. Analyzing these slices can help pinpoint specific contexts where the 'Tables' sub-category performs differently and refine the understanding of the discount's impact.

## Part H — Synthesis (DIVE: Extend)
**Aim:** Turn analysis into business‑ready insights.

### H1. Executive‑style summary
**Prompt:**
```
Act as a business strategist.
Based on the following metrics/figures (briefly summarize your results here), write a 4-sentence executive summary:
- 1 sentence: what changed and by how much
- 1 sentence: why it likely changed (drivers)
- 1 sentence: recommended action (who/what/when)
- 1 sentence: metric to monitor next
```
**Executive Summary:**
The 'Tables' sub-category showed a significant negative total profit of -64083.3887 across the dataset, indicating a substantial loss-making product line overall. This negative profitability is likely driven by high discounts applied to tables, as seen in the analysis slicing by region/time and order priority/ship mode which also show corresponding discount values alongside negative profits in various segments. To address this, the sales and product teams should immediately review the pricing and discount strategy for 'Tables', potentially adjusting prices or reducing discount levels, especially for specific regions, time periods, order priorities, or ship modes identified in the detailed analysis. The key metric to continuously monitor will be the 'Tables' sub-category's total profit and average discount percentage to track the impact of any changes.

### H2. Convert final SQL into an automated job (optional)
**Prompt (use only after your SQL is final):**
```
Convert my final BigQuery SQL into a Python script that can run as a scheduled job from Colab or Cloud Functions.
Requirements:
- Use python‑bigquery client
- Parameterize date range
- Write results to a destination table `[YOUR_PROJECT].analytics.outputs_kpi`
- Add basic error handling & logging
Return one complete runnable script.
```

---
## Submission checklist
- [ ] Kept prompts precise and reproducible  
- [ ] Captured at least **one** CTE query and **one** window function query  
- [ ] Documented **two** validation attempts (counter‑queries or alternate slice)  
- [ ] Wrote a 4‑sentence executive summary based on results  
- [ ] (Optional) Converted final query into a scheduled job
---